### API Calls from OpenFEMA

In [137]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from pprint import pprint
import requests
import json
import datetime 

In [138]:
# Load cyber security breach data set into Pandas
cyber_data = pd.read_csv("../source_data/cybersecurity_data.csv")
cyber_data.head()

,Unnamed: 0,Number,Name_of_Covered_Entity,State,Business_Associate_Involved,Individuals_Affected,Date_of_Breach,Type_of_Breach,Location_of_Breached_Information,Date_Posted_or_Updated,Summary,breach_start,breach_end,year
0,141,140,UNCG Speech and Hearing Center,NC,NaN,2300,1/1/1997,Hacking/IT Incident,Desktop Computer,1/23/2014,NaN,1/1/1997,NaN,1997
1,1030,1029,UMass Memorial Medical Center,MA,NaN,2387,5/6/2002,Unauthorized Access/Disclosure,"Electronic Medical Record, Paper",5/27/2014,NaN,5/6/2002,NaN,2002
2,212,211,Riverside Mercy Hospital and Ohio/Mercy Diagno...,OH,NaN,1000,3/29/2003,Improper Disposal,Paper,1/23/2014,NaN,3/29/2003,NaN,2003
3,272,271,SW General Inc,AZ,NaN,566,5/1/2004,Theft,Paper,1/23/2014,NaN,5/1/2004,NaN,2004
4,223,222,OhioHealth Corporation dba Grant Medical Center,OH,NaN,501,1/1/2008,Theft,"Laptop, Desktop Computer",1/23/2014,NaN,1/1/2008,NaN,2008


In [139]:
# Count the number of rows of data in data set 
len(cyber_data)

1055

In [140]:
# Extract the relevent columns from the inital data set
cyber_columns = cyber_data[["Number", "State", "year", "Date_of_Breach"]]
cyber_columns.head()

,Number,State,year,Date_of_Breach
0,140,NC,1997,1/1/1997
1,1029,MA,2002,5/6/2002
2,211,OH,2003,3/29/2003
3,271,AZ,2004,5/1/2004
4,222,OH,2008,1/1/2008


In [141]:
# Rename columns to better describe the data set
cyber_columns_renamed = cyber_columns.rename(columns={"Number": "Breach ID", "year": "Year","Date_of_Breach": "Full Date"})
cyber_columns_renamed.head()

,Breach ID,State,Year,Full Date
0,140,NC,1997,1/1/1997
1,1029,MA,2002,5/6/2002
2,211,OH,2003,3/29/2003
3,271,AZ,2004,5/1/2004
4,222,OH,2008,1/1/2008


In [142]:
# Remove rows that contain data from states outside the continental US (Alaska, Hawaii, and Puerto Rico)
# Note this data set includes DC, need to find out if wildfire data contains DC or not, kept it in cyber dataframe for now

cyber_data_clean = cyber_columns_renamed.loc[ (cyber_columns_renamed["State"] != "AK") & (cyber_columns_renamed["State"] != "HI") & (cyber_columns_renamed["State"] != "PR") ]
cyber_data_clean.head()

,Breach ID,State,Year,Full Date
0,140,NC,1997,1/1/1997
1,1029,MA,2002,5/6/2002
2,211,OH,2003,3/29/2003
3,271,AZ,2004,5/1/2004
4,222,OH,2008,1/1/2008


In [143]:
# Count number of rows of cleaned data to make sure irrelevant rows were removed
len(cyber_data_clean)

1018

In [144]:
# Write the cleaned data to a new CSV file called 'cyber_data_clean.csv'
cyber_data_clean.to_csv("../source_data/cyber_data_clean.csv")

In [145]:
#Pulling in states from data breach csv
file_path = "../source_data/cyber_data_clean.csv"
clean_df = pd.read_csv(file_path)
state_list = clean_df['State'].unique()

In [146]:
#Base URL for OpenFEMA database
base_url = f'https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries?$orderby=incidentBeginDate&$select=incidentType,state,incidentBeginDate&$inlinecount=allpages&$skip='

In [147]:
#List where disaster summaries will be stored
disas_summs = []
#Range of data limited between years 1997 and 2014
#Since the range is hard stuck between these limits, hard coding is justified
start = 15940 #Page starts with first disaster in 1997 
end = 43387 #Page ends with last disaster in 2014

In [148]:
while start < 45000: #We want the loop to end after finishing the 42,940th page 
                     #43387 - 15940 = 27447, and 15940 + 27000 = 42940  
    #Combined URL
    comb_url = base_url + str(start)
    #Grab response from URL
    response = requests.get(comb_url).json()
    #Grab list of disaster summaries from response
    summ_list = response['DisasterDeclarationsSummaries']
    #Loops through summ_list 
    for i in range(len(summ_list)):
        #Grab the correct state from response 
        given_state = summ_list[i]['state']
        #Check if the given state is in the state list (not AK, HI, or PR)
        if given_state in state_list:
            #If not one of these three, then check if the incident type contains the word Fire
            #Puts the string in incidentType to lower case before comparing strings to avoid case error
            if 'fire' in summ_list[i]['incidentType'].lower():
                #Appends following dictionary to disas_summs
                disas_summs.append(summ_list[i])
    #After appending, start is increased by 1000 to avoid summary overlap (values repeating in append) 
    start += 1000

In [47]:
#After the first 27 loops in the while loop, the last page after this must be run to get
#the remaining 2014 summaries
#This will also accidently pick up summaries from after 2014 that need to be deleted
#pg_num = 42951
#comb_url = base_url + str(pg_num)
#response = requests.get(comb_url).json()
#summ_list = response['DisasterDeclarationsSummaries'] 

In [48]:
#Loops through summ_list 
#for i in range(len(summ_list)):
#    #Grab the correct state from response 
#    given_state = summ_list[i]['state']
#    #Check if the given state is in the state list (not AK, HI, or PR)
#    if given_state in state_list:
        #If not one of these three, then check if the incident type contains the word Fire
        #Puts the string in incidentType to lower case before comparing strings to avoid case error
#        if 'fire' in summ_list[i]['incidentType'].lower():
            #Appends following dictionary to disas_summs
#            disas_summs.append(summ_list[i])

In [81]:
# creating columns for the dataframe
columns = ['dis_id', 'date', 'state', 'type']

# creating the dataframe
disaster_df = pd.DataFrame(columns = columns)

# cursing through the disaster json and appending to the new disaster_df dataframe
for i in range(len(disas_summs)):
    
    #appending the data
    disaster_df = disaster_df.append({
                  'dis_id' : disas_summs[i]['id']
                , 'date'   : disas_summs[i]['incidentBeginDate'].replace('Z', '')
                , 'state'  : disas_summs[i]['state']
                , 'type'   : disas_summs[i]['incidentType']}, ignore_index = True)
    

#exlcuding dates
disaster_df = disaster_df[(disaster_df['date'] <= '2014-12-31') & (disaster_df['date'] >= '2008-01-01')]

#making a copy
disaster_df.copy()

#converting the date
disaster_df['date'] = pd.to_datetime(disaster_df.date).dt.strftime('%Y-%m-%d')

#creating a csv
disaster_df.to_csv("../source_data/disaster_data.csv")

In [170]:
disaster_df.head()The 

,dis_id,date,state,type
1743,5f7b2c0831a8c6681cfd7421,2008-02-08,TX,Fire
1744,5f7b2c0831a8c6681cfd7423,2008-02-13,TX,Fire
1745,5f7b2c0831a8c6681cfd7426,2008-02-23,TX,Fire
1746,5f7b2c0831a8c6681cfd742a,2008-02-25,TX,Fire
1747,5f7b2c0831a8c6681cfd742e,2008-02-25,OK,Fire
...,...,...,...,...
2604,5f7b2c3231a8c6681cfffc55,2014-09-13,CA,Fire
2605,5f7b2c3231a8c6681cfffc49,2014-09-14,CA,Fire
2606,5f7b2c3231a8c6681cfffc4b,2014-09-15,CA,Fire
2607,5f7b2c3231a8c6681cfffc4e,2014-09-15,OR,Fire
